In [1]:
import requests
from aiohttp import ClientSession
import asyncio
from bs4 import BeautifulSoup
import pandas as pd

import time
import random
import logging

ModuleNotFoundError: No module named 'aiohttp'

In [2]:
logger = logging.getLogger(__name__)

In [3]:
BASIC_FORMAT = "%(asctime)s-%(levelname)s-%(message)s"
chlr = logging.StreamHandler()
chlr.setFormatter(logging.Formatter(BASIC_FORMAT))
logger.setLevel('DEBUG')
logger.addHandler(chlr)

In [4]:
class RentalCrawler:
    DOMAIN_URL = 'https://rent.591.com.tw/'
    GET_LIST_URL = 'https://rent.591.com.tw/home/search/rsList'
    HEADERS = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }

    def __init__(self):
        self.session = requests.Session()

    def _get_token(self):
        res = self.session.get(self.DOMAIN_URL, headers=self.HEADERS)
        soup = BeautifulSoup(res.text, 'html.parser')
        token = soup.select_one('meta[name="csrf-token"]').get('content')
        return token

    # def _get_rental_list(self, token: str, region_id: int=3):
    #     headers = self.HEADERS.copy()
    #     headers['X-CSRF-TOKEN'] = token
    #     c = requests.cookies.RequestsCookieJar()
    #     c.set('urlJumpIp', f'{region_id}',
    #             domain='.591.com.tw',
    #             path='/')
    #     self.session.cookies.update(c)
    #     df_rental_list = pd.DataFrame(columns=['post_id', 'title'])
    #     params = f'is_format_data=1&is_new_list=1&type=1&region={region_id}&firstRow=0'
    #     res = self.session.get(self.GET_LIST_URL, params=params, headers=headers)
    #     time.sleep(random.randint(3, 5))
    #     for house in res.json()['data']['data']:
    #         df_rental_list = df_rental_list.append({'post_id': house['post_id'], 'title': house['title']}, ignore_index=True)
    #     total_page = int(res.json()['records'].replace(',',''))//30+1
    #     if total_page > 1:
    #         for page in range(1, total_page):
    #             params = f'is_format_data=1&is_new_list=1&type=1&region={region_id}&firstRow={30*page}'
    #             res = self.session.get(self.GET_LIST_URL, params=params, headers=headers)
    #             time.sleep(random.randint(3, 5))
    #             for house in res.json()['data']['data']:
    #                 df_rental_list = df_rental_list.append({'post_id': house['post_id'], 'title': house['title']}, ignore_index=True)
    #     return df_rental_list
    def _get_rentals(self, region_id: int=3, firstRow:int=0):
        df_rentals = pd.DataFrame(columns=['post_id', 'title'])
        params = f'is_format_data=1&is_new_list=1&type=1&region={region_id}&firstRow=0'
        res = self.session.get(self.GET_LIST_URL, params=params, headers=headers)
        time.sleep(random.randint(3, 5))
        for house in res.json()['data']['data']:
            df_rental_list = df_rental_list.append({'post_id': house['post_id'], 'title': house['title']}, ignore_index=True)
        total_page = int(res.json()['records'].replace(',',''))//30+1
        if total_page > 1:
            for page in range(1, total_page):
                params = f'is_format_data=1&is_new_list=1&type=1&region={region_id}&firstRow={30*page}'
                res = self.session.get(self.GET_LIST_URL, params=params, headers=headers)
                time.sleep(random.randint(3, 5))
                for house in res.json()['data']['data']:
                    df_rental_list = df_rental_list.append({'post_id': house['post_id'], 'title': house['title']}, ignore_index=True)
        return df_rental_list

In [5]:
crawler = RentalCrawler()

In [6]:
token = crawler._get_token()

In [7]:
df_rental_list = crawler._get_rental_list(token=token, region_id=1)

In [8]:
df_rental_list

,post_id,title
0,12803640,有廚房電梯套房(押金可分2期
1,12776968,近迪化街永樂市場可工作室商辦有衛浴瓦斯
2,12802025,台北車站捷運宅/包水電/可報稅代收垃圾
3,12815672,租套房送書房東區忠孝南京復興雙捷運可貓
4,12815675,東區微風廣場2分忠孝南京復興捷運2房可貓
...,...,...
10772,12561729,大安森林公園單身女生豪華雅房
10773,12561724,"東吳,文化士林官邸單身女學生豪宅宿舍"
10774,12548076,用防火建材裝璜，辦公及学生念書環境。
10775,12543708,內有隔2大間２小間，當辨公或倉庫


In [91]:
res.json()['records']

'10,999'

In [70]:
res.json()['data']['data']

[{'title': '捷運一分鐘SOGO東區摩登優質套房',
  'type': '1',
  'post_id': 12699348,
  'kind_name': '分租套房',
  'room_str': '',
  'floor_str': '4F/4F',
  'community': '',
  'price': '16,800',
  'price_unit': '元/月',
  'photo_list': ['https://img2.591.com.tw/house/2022/05/02/165146505623750405.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2022/05/02/165146505623710908.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/28/162745585506894609.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/20/162676654472473106.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/20/162676654243873102.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/28/162745585525397502.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/07/20/162676660377755203.jpg!510x400.jpg',
   'https://img1.591.com.tw/house/2019/09/20/156896830030617105.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2019/09/20/156896830089823900.jpg!510x400.jpg',
   'https://img2.591.com.tw/house/2021/09/23/16324

In [36]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}
s = requests.Session()
url = 'https://rent.591.com.tw/'
r = s.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
token_item = soup.select_one('meta[name="csrf-token"]')

In [37]:
headers = headers.copy()
headers['X-CSRF-TOKEN'] = token_item.get('content')

In [105]:
j = requests.jar()

AttributeError: module 'requests' has no attribute 'jar'

In [ ]:
requests.cookies

In [41]:
c = requests.cookies.RequestsCookieJar()
c.set('urlJumpIp', '3',
        domain='.591.com.tw',
        path='/')
s.cookies.update(c)

Cookie(version=0, name='urlJumpIp', value='3', port=None, port_specified=False, domain='.591.com.tw', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [42]:
s.cookies.update(c)

In [44]:
s.cookies['urlJumpIp']

'3'

In [28]:
s.cookies.set('urlJumpIp', '3', domain='rent.591.com.tw', path='/')

Cookie(version=0, name='urlJumpIp', value='3', port=None, port_specified=False, domain='rent.591.com.tw', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [45]:
url = 'https://rent.591.com.tw/home/search/rsList'
# params = 'is_format_data=1&is_new_list=1&type=1&region=3&firstRow=0'
params = 'is_format_data=1&is_new_list=1&type=1&region=3'
r = s.get(url, params=params, headers=headers)

In [84]:
# https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&region=3
r = s.get('', params=params, headers=headers)

In [58]:
r.json().keys()

dict_keys(['status', 'data', 'records', 'is_recom', 'deal_recom', 'online_social_user', 'bluekai_data', 'recommend', 'seo'])

In [70]:
int(r.json()['records'].replace(',',''))//30+1

254

In [69]:
1//30

0

In [55]:
df_rental_list = pd.DataFrame(columns=['post_id', 'title'])
for house in r.json()['data']['data']:
    df_rental_list = df_rental_list.append({'post_id': house['post_id'], 'title': house['title']}, ignore_index=True)

In [56]:
df_rental_list

,post_id,title
0,12750791,環球莒光寵物友善兩大房格局
1,12713709,環球莒光寵物友善兩大房格局
2,12815016,近頂溪捷運站精心裝修的二楼舒適套房
3,12696888,【1樓整層】及【2、3樓女性套房】出租
4,12565749,板橋民生花市橋下停車場平日月租
5,12632769,新台五路二段套房出租
6,12684586,獨立衛浴~近捷運~可報稅申請補貼~限女性
7,12764172,台北矽谷大樓地下停車場
8,12805436,板橋.府中.對外窗.獨洗.台藝大致理.
9,12759355,【新大樓】永安捷運旁、陽台套房、中和路


In [50]:
r.json()['data']['data'][0]

{'title': '環球莒光寵物友善兩大房格局',
 'type': '1',
 'post_id': 12750791,
 'kind_name': '整層住家',
 'room_str': '2房1廳',
 'floor_str': '5F/5F',
 'community': '',
 'price': '24,800',
 'price_unit': '元/月',
 'photo_list': ['https://img1.591.com.tw/house/2022/06/14/165520490972242303.jpg!510x400.jpg',
  'https://img1.591.com.tw/house/2022/06/14/165520416894674101.jpg!510x400.jpg',
  'https://img2.591.com.tw/house/2022/06/14/165520416968016308.jpg!510x400.jpg',
  'https://img1.591.com.tw/house/2022/06/14/165520417031995408.jpg!510x400.jpg',
  'https://img1.591.com.tw/house/2022/06/14/165520417064122303.jpg!510x400.jpg'],
 'section_name': '中和區',
 'street_name': '國光街',
 'location': '中和區-國光街107巷',
 'rent_tag': [{'id': '1', 'name': '屋主直租'},
  {'id': '3', 'name': '拎包入住'},
  {'id': '5', 'name': '隨時可遷入'},
  {'id': '6', 'name': '可開伙'},
  {'id': '7', 'name': '可養寵物'}],
 'area': '25',
 'role_name': '屋主',
 'contact': '林先生',
 'refresh_time': '9分鐘內',
 'yesterday_hit': 326,
 'is_vip': 1,
 'is_combine': 1,
 'hurry': 0,
 

In [9]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}
s = requests.Session()
url = f'https://rent.591.com.tw/home/12670629'
r = s.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
token_item = soup.select_one('meta[name="csrf-token"]')

headers = headers.copy()
headers['X-CSRF-TOKEN'] = token_item.get('content')
headers['deviceid'] = s.cookies.get_dict()['T591_TOKEN']
# headers['token'] = s.cookies.get_dict()['PHPSESSID']
headers['device'] = 'pc'

url = f'https://bff.591.com.tw/v1/house/rent/detail?id=12670629'
r = s.get(url, headers=headers)

In [11]:
r.json()['data']

{'breadcrumb': [{'name': '台北市',
   'id': 1,
   'query': 'region',
   'link': '/?region=1'},
  {'name': '萬華區', 'id': 6, 'query': 'section', 'link': '/?region=1&section=6'},
  {'name': '分租套房',
   'id': 3,
   'query': 'kind',
   'link': '/?region=1&section=6&kind=3'}],
 'title': '西門新套房可炊有烘衣機廚房',
 'deposit': '押金二個月',
 'kind': 3,
 'relieved': 0,
 'regionId': 1,
 'sectionId': 6,
 'shareInfo': {'url': 'https://www.591.com.tw/1R?salt=BYsK&s=',
  'from': '來自【591租屋】',
  'title': '台北市萬華區，分租套房出租，12867元/月，詳情：'},
 'dealText': '',
 'dealTime': 0,
 'browse': {'pc': 70, 'mobile': 161},
 'tags': [{'id': 2, 'value': '近捷運'},
  {'id': 3, 'value': '拎包入住'},
  {'id': 4, 'value': '近商圈'},
  {'id': 5, 'value': '隨時可遷入'},
  {'id': 6, 'value': '可開伙'}],
 'price': '12,867',
 'priceUnit': '元/月',
 'navData': [{'title': '位置與周邊', 'key': 'positionRound', 'active': 1},
  {'title': '設備與服務', 'key': 'service', 'active': 1},
  {'title': '屋況介紹', 'key': 'remark', 'active': 1},
  {'title': '房屋詳情', 'key': 'houseDetail', 'active': 